In [1]:
import gradio as gr
import torch
import pickle
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import re

# ----------------------------
# Load Text Moderation Model
# ----------------------------
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

with open("hashing_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

with open("text_moderation_model.pkl", "rb") as f:
    text_model = pickle.load(f)

# ----------------------------
# Load Image Moderation Model (ResNet18)
# ----------------------------
resnet_model = resnet18(weights=ResNet18_Weights.DEFAULT)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 3)
resnet_model.load_state_dict(torch.load("resnet_best.pth", map_location=torch.device("cpu")))
resnet_model.eval()

image_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ----------------------------
# Moderation Storage
# ----------------------------
flagged_posts = []
allowed_posts = []
blocked_posts = []
admin_password = "admin123"

# ----------------------------
# Moderation Logic
# ----------------------------
def moderate_post(text, image):
    clean_text = preprocess_text(text)
    text_features = vectorizer.transform([clean_text])
    text_prediction = text_model.predict(text_features)[0]

    if image is None:
        return "No image uploaded"

    temp_path = "temp_uploaded_image.jpg"
    if isinstance(image, np.ndarray):
        pil_img = Image.fromarray(image.astype('uint8'), 'RGB')
        pil_img.save(temp_path)
    elif isinstance(image, Image.Image):
        image.save(temp_path)
    else:
        return "Unsupported image format"

    image = Image.open(temp_path).convert("RGB")
    image_input = image_transform(image).unsqueeze(0)

    with torch.no_grad():
        outputs = resnet_model(image_input)
        probs = F.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probs, 1)
        image_prediction = predicted.item()

    image_labels = {0: "Neutral", 1: "NSFW", 2: "Violence"}

    if image_prediction != 0 and probs[0][0] > 0.8:
        image_prediction = 0

    if text_prediction == 0 and image_prediction == 0:
        allowed_posts.append((text, image))
        return "Post Allowed"
    elif text_prediction == 1 and image_prediction in [1, 2]:
        blocked_posts.append((text, image))
        return "Post Blocked"
    else:
        flagged_posts.append((text, image))
        return f"Post Flagged for Admin Review (Image: {image_labels[image_prediction]})"

# ----------------------------
# Admin Functions
# ----------------------------
def admin_access(password):
    if password == admin_password:
        if not flagged_posts:
            return "No flagged posts."
        return [f"#{i}: {flag[0][:60]}..." for i, flag in enumerate(flagged_posts)]
    else:
        return "Incorrect Password"

def allow_flagged(index):
    index = int(index)
    if 0 <= index < len(flagged_posts):
        allowed_posts.append(flagged_posts[index])
        flagged_posts.pop(index)
        return f"Flagged Post #{index} Allowed"
    return "Invalid Index"

def block_flagged(index):
    index = int(index)
    if 0 <= index < len(flagged_posts):
        blocked_posts.append(flagged_posts[index])
        flagged_posts.pop(index)
        return f"Flagged Post #{index} Blocked"
    return "Invalid Index"

# ----------------------------
# Gradio Interface with Design
# ----------------------------
with gr.Blocks(css="""
    .blue-button {background-color: #007bff !important; color: white;}
    .green-button {background-color: #28a745 !important; color: white;}
    .red-button {background-color: #dc3545 !important; color: white;}
    .center-box {display: flex; flex-direction: column; align-items: center; justify-content: center; max-width: 400px; margin: auto;}
    .login-heading {font-size: 24px; font-weight: bold; color: #6c2dc7; margin-bottom: 10px;}
""") as demo:

    gr.Markdown("## 🛡️ Social Media Content Moderation System")

    with gr.Tab(" User Post"):
        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload Image", type="pil")
                text_input = gr.Textbox(label="Enter Text", placeholder="Type your post here...", lines=4)
                submit_btn = gr.Button("Submit Post", elem_classes=["blue-button"])
            with gr.Column():
                output = gr.Textbox(label="Moderation Result", lines=4)

        submit_btn.click(fn=moderate_post, inputs=[text_input, image_input], outputs=output)

    with gr.Tab(" Admin Panel"):
        with gr.Column(elem_classes=["center-box"]):
            gr.Markdown(" <span class='login-heading'>Admin Login</span>")
            password_input = gr.Textbox(label="Password", type="password", placeholder="Enter admin password...")
            access_btn = gr.Button("Access Flagged Posts", elem_classes=["blue-button"])
            flagged_output = gr.Textbox(label="Flagged Posts", lines=6)

        index_input = gr.Number(label="Flagged Post Index (e.g., 0, 1, 2)")

        with gr.Row():
            allow_btn = gr.Button("Allow", elem_classes=["green-button"])
            block_btn = gr.Button("Block", elem_classes=["red-button"])

        result_box = gr.Textbox(label="Action Result")

        access_btn.click(fn=admin_access, inputs=password_input, outputs=flagged_output)
        allow_btn.click(fn=allow_flagged, inputs=index_input, outputs=result_box)
        block_btn.click(fn=block_flagged, inputs=index_input, outputs=result_box)


demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
